In [6]:
#generate graphs with chunk_size, batch size
import psycopg2
import pandas as pd
import nltk
import re #This is to parse the HTML code from the given text
from sshtunnel import SSHTunnelForwarder #This is to connect to the local Database

def getCharAuthorData(authors, doc, documentTable = 'document_unicode', chunk_size = 1000):
    df = pd.DataFrame()
    conn = None
    output = []
    i = 1
    #nltk.download('punkt')
    try:
        
        with SSHTunnelForwarder(("srn01.cs.cityu.edu.hk", 22),
                            ssh_username='stylometry',
                            ssh_password='stylometry',
                            remote_bind_address=('localhost', 5432),
                            local_bind_address=('localhost', 5400)):
            
            conn = psycopg2.connect(user="stylometry", password="stylometry",
                                database="stylometry", host="localhost", port=5400)
            
            
            cur = conn.cursor()
            query = "SELECT author_id, doc_content FROM " + str(documentTable) + " WHERE author_id IN ("
            flag = False
            for auth in authors:
                if not flag:
                    query = query + str(auth)
                    flag = True
                else:
                    query = query + ", " + str(auth)
            query = query + ") AND doc_id <> '" + str(doc) + "' ;"
            cur.execute(query)
            print("Execution completed")
            rows = cur.fetchall()
            
            print("Read completed")
            print("Number of rows: %s" % (len(rows)))
            for row in rows:
                #tokens = nltk.word_tokenize(row[1])
                ###row[1] = row[1].replace("\r\n","")
                
                temp = re.sub('<[^<]+?>', '', row[1])
                temp = temp.replace("\r\n","") 
                temp = temp.replace("\n","") 
                chars = list(temp)
                
                chunk1 = []
                for x in chars:
                    if (i < chunk_size):
                        chunk1.append(x)
                        i += 1
                    else:
                        chunk1.append(x)
                        xx = ''.join(chunk1)
                        xx = str(xx)
                        chunk1 = []
                        output.append([row[0], xx])
                        i = 1
                if len(chunk1) > 0:
                    xx = ''.join(chunk1)
                    xx = str(xx)
                    chunk1 = []
                    output.append([row[0], xx])
                    i = 1

            df = pd.DataFrame(output, columns=["author_id", "doc_content"])
            print(df.dtypes)
            print("Data Frame created: Shape: %s" % (str(df.shape)))

    except psycopg2.Error as e:
        if conn:
            conn.rollback()
        print('Error %s' % e)
        sys.exit(1)

    finally:
        if conn is not None:
            conn.close()
    
    print df
    return df



In [7]:

authors=[ 1240, 172 ]
doc = 61
df = getCharAuthorData(authors, doc, documentTable = 'document_unicode', chunk_size = 1000)


Execution completed
Read completed
Number of rows: 2
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (167, 2)
     author_id                                        doc_content
0          172  This eBook was prepared by Gunther Olesch from...
1          172  rt ihn.]HAUPTMANN: Langsam, Woyzeck, langsam; ...
2          172  icht, ein guter Mensch, der sein gutes Gewisse...
3          172  ort? Ermacht mich ganz konfus mit seiner Antwo...
4          172  dieNatur; aber wenn ich ein Herr wär und hätt'...
5          172  ne, grüne Gras...WOYZECK: Still: Hörst du's, A...
6          172  s ein Mann, wie ein Baum!MARIE: Er steht auf s...
7          172   du's, Franz? Komm herein!WOYZECK: Kann nit. M...
8          172  erben,        das ist uns wohlbekannt.WOYZECK:...
9          172   Willst Du?MARIE: Meinetwegen. Das muß schön D...
10         172  ernen. -Das war einfacher Verstand. Denk jetzt...
11         172   [Zieht großartig und gemessen eine Uhr ausde

In [ ]:
1458